In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import os
import os.path
import pandas as pd
import numpy as np
import sys
import re
import sys
import json
from datetime import datetime
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

pd.set_option('display.max_columns', 50)

In [33]:
from cig_script.config.config_env import Config

from cig_script.mail_copy_input_format import _get_one_mail_dict
from cig_script.mail_copy_input_format import _get_mul_mail_dict
from cig_script.mail_copy_input_format import _get_all_sheet_from_mail
from cig_script.mail_copy_input_format import _check_include_df

# from cig_script.mail_copy_input_format import _autowidth_excel
# from datetime import datetime





In [24]:
mail_copy_in_path = Config().mail_copy_in_path
mul_sheet_list = []
one_sheet_list = []
one_result_dict = {}
mul_result_dict = {}


In [25]:
one_sheet_list,mul_sheet_list = _get_all_sheet_from_mail(mail_copy_in_path,one_sheet_list,mul_sheet_list)

# print('Do you want to merge current mail info in TOTAL sheet?')
# total_option = input('1 : Y(Yes) or  0: N(No) : ')

if one_sheet_list and mul_sheet_list:
    one_result_dict = _get_one_mail_dict(mail_copy_in_path,one_result_dict,one_sheet_list)
    mul_result_dict = _get_mul_mail_dict(mail_copy_in_path,mul_result_dict,mul_sheet_list)
elif one_sheet_list:
    one_result_dict = _get_one_mail_dict(mail_copy_in_path,one_result_dict,one_sheet_list)
elif mul_sheet_list:
    mul_result_dict = _get_mul_mail_dict(mail_copy_in_path,mul_result_dict,mul_sheet_list)
else:
    pass

In [26]:
tot_mail_sheet = Config().total_mail_sheet_name
tot_excel_from_mail = Config().tot_excel_from_mail

In [27]:
total_sheet_df = pd.read_excel(tot_excel_from_mail,sheet_name=tot_mail_sheet)

In [28]:
total_sheet_df

,CONSIGNEE NAME,MODEL,YR,CHASSINO,ACID NO,IMPORTER TAX NUMBER,FREIGHT FORWARDER ID,SHEET,gen_time
0,SALEH AHMED MOHAMED SALEH,K3,2013,KNAFK412BDA050028,9992022070000061164,999999999,950449415,ELMONGY,2022-08-30 21:19
1,NABIELAH MAGDY AATTIA MOHAMED AATTIA HELAL,AVANTE,2010,KMHDU41BBAU098545,9992022070000038337,999999999,940426900,ELMONGY,2022-08-30 21:19
2,BASSIOUNY MOHAMED HAFEZ MOHAMED ELSHAFAAY,FORTE,2009,KNAFH22139A014476,9992022070000061172,999999999,950449415,ELMONGY,2022-08-30 21:19
3,REDA MAHMOUD SADEK MOUSTAFA,AVANTE,2016,KMHD741DBGU173749,9992022080000014089,999999999,903204053,YOUSEF,2022-08-30 21:19
4,AASEM MOHAMED ABD ELFATAH ABD ELRAHMAN,K3,2013,KNAFK412BDA083247,9992022080000014329,999999999,903204053,YOUSEF,2022-08-30 21:19
5,AAWAAD ABD ELAATY MOHAMED EBRAHIM,TRAX,2016,KLABA76BDGB748116,9992022080000014402,999999999,903204053,YOUSEF,2022-08-30 21:19
6,MOHAMED SHAABAN KHALF HUSSIEN,FORTE,2009,KNAFH22139A050074,9992022080000014485,999999999,903204053,YOUSEF,2022-08-30 21:19
7,MOHAMED BAHAG ELDIEN AHMED HUSSIEN,AVANTE,2011,KMHDG41DBBU137819,9992022080000014667,999999999,903204053,YOUSEF,2022-08-30 21:19
8,AHMED MOHAMED ELAAWADY ABD ELAAL,AVANTE,2011,KMHDG41DBBU024372,9992022070000061545,999999999,950496286,SAM,2022-08-30 21:19
9,WALAA GAMIEL FATHY SEDIEK,K3,2013,KNAFZ412BDA004986,9992022070000061487,999999999,950496286,SAM,2022-08-30 21:19


In [44]:
now = datetime.now()
gen_time = now.strftime('%Y-%m-%d %H:%M')

if one_result_dict:
    _total_one_df = pd.DataFrame()
    for _sheet,_car_df in one_result_dict.items():
        _car_df['gen_time'] = gen_time
        _car_df['SHEET'] = _sheet
        _total_one_df = pd.concat([_total_one_df,_car_df])
    
    _is_include_one_car,exist_dup_one_car,sub_one_car_df = _check_include_df(_total_one_df,total_sheet_df,gen_time)
    print(_is_include_one_car)
    if _is_include_one_car:
        pass
    else:
        if exist_dup_one_car:
            if sub_one_car_df.empty:
                total_sheet_df = total_sheet_df.set_index('YR').reset_index()
            else:
                total_sheet_df = pd.concat([total_sheet_df,sub_one_car_df]).set_index('YR').reset_index()
        else:
            total_sheet_df = pd.concat([total_sheet_df,_car_df]).set_index('YR').reset_index()

True


In [45]:
if mul_result_dict:
    _total_mul_df = pd.DataFrame()
    for _sheet,_car_df in mul_result_dict.items():
        _car_df['gen_time'] = gen_time
        _car_df['SHEET'] = _sheet
        _total_mul_df = pd.concat([_total_mul_df,_car_df])
    
#     print(total_sheet_df)
    _is_include_mul_car,exist_dup_mul_car,sub_mul_car_df = _check_include_df(_total_mul_df,total_sheet_df,gen_time)

    if _is_include_mul_car:
        pass
    else:
        if exist_dup_mul_car:
            if sub_mul_car_df.empty:
                total_sheet_df = total_sheet_df.set_index('YR').reset_index()
            else:
                total_sheet_df = pd.concat([total_sheet_df,sub_mul_car_df]).set_index('YR').reset_index()
        else:
            total_sheet_df = pd.concat([total_sheet_df,_car_df]).set_index('YR').reset_index()

In [47]:
if one_result_dict or mul_result_dict:
    if not os.path.exists(tot_excel_from_mail):
        print('new')
    else:
        print(_is_include_one_car,_is_include_mul_car)
        if _is_include_one_car and _is_include_mul_car:
            print('both include already')
            pass
        elif _is_include_one_car:
            print('mul car update')
            print(exist_dup_mul_car)
        elif _is_include_mul_car:
            print('mul car update')

True True
